In [48]:
import pandas as pd
# import pandas_ta as ta
import time
from moexalgo import Market, Ticker, CandlePeriod
from datetime import datetime, timedelta
import requests

In [49]:
import moexalgo
moexalgo.__version__

'2.0.3'

In [43]:
# У кого 2.0.3 версия

#  нефть [32:43]
# юань [62:67]
# евро / доллар 80:82
# евро/ рубль 91:94
# золото 107:110
# газ  219:224
# рубль/Доллар 313:320
# серебро 337:340

fo = Market('FO')
fo = fo.tickers()
fo = pd.DataFrame(fo)

In [50]:
def send_telegram(text: str):
    token = "7724948126:AAE7vzbSwnUNWWszFsvUDSPKorR4jphXYOs"
    url = "https://api.telegram.org/bot"
    channel_id = "-1002312259096"
    url += token
    method = url + "/sendMessage"

    r = requests.post(method, data={
        "chat_id": channel_id,
        "text": text
    })

    if r.status_code != 200:
        raise Exception("post_text error")

In [7]:
def timer_func(delta_hours=600):
    # Получаем текущую дату и время
    current_date = datetime.now()

    # Создаем дельту в 600 часов
    delta = timedelta(hours=delta_hours)
    
    # Вычитаем дельту из текущей даты
    past_date = current_date - delta
    
    # Преобразуем даты в строки в формате "день-месяц-год"
    current_date_str = current_date.strftime("%Y-%m-%d")
    past_date_str = past_date.strftime("%Y-%m-%d")
    
    return past_date_str, current_date_str

In [ ]:
# Функция для обработки данных свечей
def process_candles(data, ticker_name):
    df = pd.DataFrame(data)
    # df.drop(columns=['end'], inplace=True)
    df.rename(columns={'begin': 'date'}, inplace=True)
    # df.set_index('date', inplace=True)
    # df['ticker'] = ticker_name
    return df

In [11]:
def upper_strat(df, threshold=0.003):
    """
    Стратегия повышения цены (upper_strat).
    
    Сравнивает текущую цену закрытия с предыдущей.
    Если прирост цены больше или равен заданному порогу (threshold), возвращает True и значение прироста.
    В противном случае возвращает False и значение прироста.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param threshold: Порог прироста цены.
    :return: Кортеж (булево значение, прирост цены).
    """
    initial_price = df['close'].iloc[-1] # Последняя наша цена
    prev_price = df['close'].iloc[-2] # Предыдущая цена от последней цены
    if (initial_price - prev_price) / prev_price >= threshold:
        return True, (initial_price - prev_price) / prev_price  # Возвращаем True и изменение цены
    return False, (initial_price - prev_price) / prev_price # Возвращаем False и изменение цены

def down_strat(df, threshold=0.003):
    """
    Стратегия понижения цены (down_strat).
    
    Сравнивает текущую цену закрытия с предыдущей.
    Если снижение цены больше или равно заданному порогу (threshold), возвращает True и значение снижения.
    В противном случае возвращает False и значение снижения.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param threshold: Порог снижения цены.
    :return: Кортеж (булево значение, снижение цены).
    """
    initial_price = df['close'].iloc[-1] # Последняя наша цена
    prev_price = df['close'].iloc[-2] # Предыдущая цена от последней цены
    if (prev_price - initial_price) / prev_price >= threshold:
        return True, (prev_price - initial_price) / prev_price  # Возвращаем True и индекс времени, когда произошло падение
    return False, (prev_price - initial_price) / prev_price

def ema_strat(df, short_period = 10, long_period = 40):
    """
    Стратегия на основе экспоненциальных скользящих средних (EMA).
    
    Сравнивает короткую и длинную EMA.
    Если короткая EMA пересекает длинную EMA снизу вверх, возвращает сигнал "Покупка".
    Если короткая EMA пересекает длинную EMA сверху вниз, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param short_period: Период для короткой EMA.
    :param long_period: Период для длинной EMA.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy() # Копируем наш основной датафрейм по выбранному тикеру
    df_copy[f'EMA_{short_period}'] = ta.ema(df_copy['close'], length=short_period) # Создаем через Pandas_ta короткую EMA
    df_copy[f'EMA_{long_period}'] = ta.ema(df_copy['close'], length=long_period) # Создаем через Pandas_ta длинную EMA
    
    if df_copy[f'EMA_{short_period}'].iloc[-1] > df_copy[f'EMA_{long_period}'].iloc[-1] and \
    df_copy[f'EMA_{short_period}'].iloc[-2] <= df_copy[f'EMA_{long_period}'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy[f'EMA_{short_period}'].iloc[-1] < df_copy[f'EMA_{long_period}'].iloc[-1] and \
    df_copy[f'EMA_{short_period}'].iloc[-2] >= df_copy[f'EMA_{long_period}'].iloc[-2]:
        return 'Продажа'
    
    else:
        return None
    
def range_strat(df, long_period = 120):
    """
    Стратегия на основе простой скользящей средней (SMA).
    
    Сравнивает текущую цену закрытия с длинной SMA.
    Если цена пересекает SMA снизу вверх, возвращает сигнал "Покупка".
    Если цена пересекает SMA сверху вниз, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param long_period: Период для длинной SMA.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy[f'SMA_{long_period}'] = ta.sma(df_copy['close'], length=long_period)
    
    if df_copy['close'].iloc[-1] > df_copy[f'SMA_{long_period}'].iloc[-1] and \
    df_copy['close'].iloc[-2] <= df_copy[f'SMA_{long_period}'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy['close'].iloc[-1] < df_copy[f'SMA_{long_period}'].iloc[-1] and \
    df_copy['close'].iloc[-2] >= df_copy[f'SMA_{long_period}'].iloc[-2]:
        return 'Продажа'
    else:
        return None
    
def rsi_strat(df, period=14, overbought_threshold=70, oversold_threshold=30):
    """
    Стратегия, основанная на индексе относительной силы (RSI).
    
    Если RSI превышает уровень перекупленности (overbought_threshold), возвращается сигнал "Продажа".
    Если RSI опускается ниже уровня перепроданности (oversold_threshold), возвращается сигнал "Покупка".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param period: Период для расчета RSI.
    :param overbought_threshold: Уровень перекупленности для RSI.
    :param oversold_threshold: Уровень перепроданности для RSI.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy['RSI'] = ta.rsi(df_copy['close'], length=period)
    
    if df_copy['RSI'].iloc[-1] > overbought_threshold and df_copy['RSI'].iloc[-2] <= overbought_threshold:
        return 'Продажа'
    elif df_copy['RSI'].iloc[-1] < oversold_threshold and df_copy['RSI'].iloc[-2] >= oversold_threshold:
        return 'Покупка'
    else:
        return None
    
def bollinger_bands_strat(df, period=20, std_dev=2):
    """
    Стратегия на основе индикатора полос Боллинджера (Bollinger Bands).
    
    Сравнивает текущую цену закрытия с верхней и нижней полосами Боллинджера.
    Если цена закрытия ниже нижней полосы Боллинджера, возвращает сигнал "Покупка".
    Если цена закрытия выше верхней полосы Боллинджера, возвращает сигнал "Продажа".
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param period: Период для расчета SMA и полос Боллинджера.
    :param std_dev: Количество стандартных отклонений для расчета полос Боллинджера.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    df_copy['SMA'] = ta.sma(df_copy['close'], length=period)
    df_copy['Upper_Band'] = df_copy['SMA'] + (df_copy['close'].rolling(window=period).std() * std_dev)
    df_copy['Lower_Band'] = df_copy['SMA'] - (df_copy['close'].rolling(window=period).std() * std_dev)
    
    if df_copy['close'].iloc[-1] < df_copy['Lower_Band'].iloc[-1] and df_copy['close'].iloc[-2] >= df_copy['Lower_Band'].iloc[-2]:
        return 'Покупка'
    
    elif df_copy['close'].iloc[-1] > df_copy['Upper_Band'].iloc[-1] and df_copy['close'].iloc[-2] <= df_copy['Lower_Band'].iloc[-2]:
        return 'Продажа'
    else:
        return None
    
def macd_rsi_strat(df, macd_short=12, macd_long=26, macd_signal=9, rsi_period=14, rsi_overbought=70, rsi_oversold=30):
    """
    Стратегия на основе комбинации индикаторов MACD и RSI.
    
    Сравнивает сигналы MACD и RSI.
    - Покупка, если MACD пересекает сигнальную линию снизу вверх и RSI ниже уровня перепроданности.
    - Продажа, если MACD пересекает сигнальную линию сверху вниз и RSI выше уровня перекупленности.
    
    :param df: DataFrame с колонкой 'close', содержащей цены закрытия.
    :param macd_short: Короткий период для MACD.
    :param macd_long: Длинный период для MACD.
    :param macd_signal: Период сигнальной линии MACD.
    :param rsi_period: Период для расчета RSI.
    :param rsi_overbought: Уровень перекупленности для RSI.
    :param rsi_oversold: Уровень перепроданности для RSI.
    :return: Сигнал "Покупка", "Продажа" или None.
    """
    df_copy = df.copy()
    
    # Вычисляем MACD
    macd = ta.macd(df_copy['close'], fast=macd_short, slow=macd_long, signal=macd_signal)
    df_copy['MACD'] = macd[f'MACD_{macd_short}_{macd_long}_{macd_signal}']
    df_copy['MACD_Signal'] = macd[f'MACDs_{macd_short}_{macd_long}_{macd_signal}']

    
    # Вычисляем RSI
    df_copy['RSI'] = ta.rsi(df_copy['close'], length=rsi_period)
    
    # Сигнал на покупку
    if df_copy['MACD'].iloc[-1] > df_copy['MACD_Signal'].iloc[-1]  \
    and df_copy['MACD'].iloc[-2] <= df_copy['MACD_Signal'].iloc[-2] \
    and df_copy['RSI'].iloc[-1] < rsi_oversold:
        return 'Покупка'
    
    # Сигнал на продажу
    elif df_copy['MACD'].iloc[-1] < df_copy['MACD_Signal'].iloc[-1] \
    and df_copy['MACD'].iloc[-2] >= df_copy['MACD_Signal'].iloc[-2] \
    and df_copy['RSI'].iloc[-1] > rsi_overbought:
        return 'Продажа'
    
    else:
        return None

In [12]:
def upper_strat_volume(df, threshold=0.07):
    """
    Стратегия на памп Объемов
    ...
    """
    initial_price = df['volume'].iloc[-1] # Последняя наша цена
    prev_price = df['volume'].iloc[-2] # Предыдущая цена от последней цены
    if (initial_price - prev_price) / prev_price >= threshold:
        return True, (initial_price - prev_price) / prev_price  # Возвращаем True и изменение цены
    return False, (initial_price - prev_price) / prev_price # Возвращаем False и изменение цены

In [46]:
moexalgo_stock = ['BRG5','CRH5','EDH5', 'EuH5', 'GDH5','NGG5', 'SiH5', 'SVH5']

In [47]:
while True:
    past_date_str, current_date_str = timer_func() # Функция для записи стартовой и конечной даты
    signals = 0 # Счетчик всех сигналов за прогон бота
    print('\n')
    print(f"Стартовая дата: {past_date_str}, Конечная дата: {current_date_str}")
    print('\n')
    
    try:
        for ticker in moexalgo_stock: # Пишем цикл для обработки всех тикеров в нашем листе
            data = Ticker(ticker).candles(start=past_date_str, end=current_date_str, period=60) # 60 == ЧАС
            data = process_candles(data, ticker) # Преобразоваие с колонками
            
            if upper_strat_volume(data, threshold=0.07)[0] == True:
                print(f"Тикер: {ticker}, Повышение ОБЪЕМА на {round(upper_strat_volume(data, threshold=0.07)[1],4)}, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Повышение ОБЪЕМА на {round(upper_strat_volume(data, threshold=0.07)[1],4)}, Цена: {data['close'].iloc[-1]}")             
                signals +=1

            if upper_strat(data, threshold=0.003)[0] == True:
                print(f"Тикер: {ticker}, Повышение цены на {round(upper_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Повышение цены на {round(upper_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if down_strat(data, threshold=0.003)[0] == True:
                print(f"Тикер: {ticker}, Понижение цены на {round(down_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Понижение цены на {round(down_strat(data, threshold=0.003)[1],4)}, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if ema_strat(data, short_period = 10, long_period = 40) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if ema_strat(data, short_period = 10, long_period = 40) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по двум ЕМА, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по двум ЕМА, Цена: {data['close'].iloc[-1]}")      
                signals +=1
                
            if range_strat(data, long_period = 120) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по SMA, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по SMA, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if range_strat(data, long_period = 120) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по SMA, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по SMA, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if rsi_strat(data, period=14, overbought_threshold=70, oversold_threshold=30) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по RSI, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по RSI, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if rsi_strat(data, period=14, overbought_threshold=70, oversold_threshold=30) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по RSI, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по RSI, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if bollinger_bands_strat(data, period=20, std_dev=2) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по BBands, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по BBands, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if bollinger_bands_strat(data, period=20, std_dev=2) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по BBands, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по BBands, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if macd_rsi_strat(data, macd_short=12, macd_long=26, macd_signal=9, rsi_period=14, rsi_overbought=70, rsi_oversold=30) == 'Покупка':
                print(f"Тикер: {ticker}, Покупка по Macd_Rsi, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Покупка по Macd_Rsi, Цена: {data['close'].iloc[-1]}")
                signals +=1
                
            if macd_rsi_strat(data, macd_short=12, macd_long=26, macd_signal=9, rsi_period=14, rsi_overbought=70, rsi_oversold=30) == 'Продажа':
                print(f"Тикер: {ticker}, Продажа по Macd_Rsi, Цена: {data['close'].iloc[-1]}")
                send_telegram(f"Тикер: {ticker}, Продажа по Macd_Rsi, Цена: {data['close'].iloc[-1]}")
                signals +=1
    except:
        print(f"Ошибка по тикеру: {ticker}")
        moexalgo_stock.remove(ticker)
    
    print(f'\nПрогон завершен\nСигналов: {signals}')
    time.sleep(3600) # для часа 
     # time.sleep(5) # для теста 5 секунд 













Стартовая дата: 2024-12-24, Конечная дата: 2025-01-18


Тикер: CRH5, Повышение ОБЪЕМА на 0.6567, Цена: 14.238
Тикер: EDH5, Повышение ОБЪЕМА на 0.2326, Цена: 1.0218
Тикер: NGG5, Продажа по двум ЕМА, Цена: 3.472
Тикер: SiH5, Повышение ОБЪЕМА на 0.4037, Цена: 105305.0
Тикер: SVH5, Повышение ОБЪЕМА на 0.5487, Цена: 31.42

Прогон завершен
Сигналов: 5


Стартовая дата: 2024-12-24, Конечная дата: 2025-01-18


Тикер: CRH5, Повышение ОБЪЕМА на 0.6567, Цена: 14.238
Тикер: EDH5, Повышение ОБЪЕМА на 0.2326, Цена: 1.0218
Тикер: NGG5, Продажа по двум ЕМА, Цена: 3.472
Тикер: SiH5, Повышение ОБЪЕМА на 0.4037, Цена: 105305.0
Тикер: SVH5, Повышение ОБЪЕМА на 0.5487, Цена: 31.42

Прогон завершен
Сигналов: 5


Стартовая дата: 2024-12-24, Конечная дата: 2025-01-18


Тикер: CRH5, Повышение ОБЪЕМА на 0.6567, Цена: 14.238
Тикер: EDH5, Повышение ОБЪЕМА на 0.2326, Цена: 1.0218
Тикер: NGG5, Продажа по двум ЕМА, Цена: 3.472
Тикер: SiH5, Повышение ОБЪЕМА на 0.4037, Цена: 105305.0
Тикер: SVH5, Повышение ОБЪЕМА на

KeyboardInterrupt: 